In [16]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

from tensorflow.keras.layers import Dense
from keras_tuner import RandomSearch, Hyperband
from keras.callbacks import ModelCheckpoint

import pandas as pd 

In [17]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("Starter_Code/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [18]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df= application_df.drop(["EIN", "NAME"], axis = 1)

In [19]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [20]:
# Look at APPLICATION_TYPE value counts for binning
application_counts=application_df["APPLICATION_TYPE"].value_counts()

In [21]:
application_types_to_replace = (list(application_counts[application_counts < 500].index))
application_types_to_replace

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [22]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [23]:
class_count = application_df["CLASSIFICATION"].value_counts()

In [24]:
class_count[class_count > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [25]:
# Determine which values to replace if counts are less than 1000
classifications_to_replace = list(class_count[class_count < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [26]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [27]:
# Convert categorical data to numeric with `pd.get_dummies`
dummified_df = pd.get_dummies(application_df, dtype=int)
# categorical_columns = application_df.select_dtypes(include=["object"]).columns

In [28]:
# pd.get_dummies(categorical_columns, dtype="int")

In [29]:
# Split our preprocessed data into our features and target arrays
X = dummified_df.drop("IS_SUCCESSFUL", axis=1)
y = dummified_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [30]:
# method to optimize model with hyperparameter options
def create_model(hp):
    model = tf.keras.models.Sequential()
    
    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice("activation", ["relu", "tanh", "sigmoid"])
    
    # Allow kerastuner to decide number of neurons in first layer
    model.add(Dense(units = hp.Int("num_neuron", 1,10, step = 3),
                   activation = activation,
                    input_dim = X_train.shape[1]
                   ))
    
     # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for x in range(hp.Int("num_layers", 1,6)):
        model.add(Dense(units = hp.Int("num_layers_" + str(x + 2), 1, 10, step = 2),
                       activation = activation
                       ))
        
    model.add(Dense(units=1, activation = "sigmoid"))
    
    # Compile the model
    model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    return model

In [31]:
tuner_hyperband = Hyperband(
                    create_model,
                    objective= "val_accuracy",
                    max_epochs= 10,
                    hyperband_iterations=2,
                    overwrite = True
                    )

In [32]:
# Run the kerastuner search for best hyperparameters
tuner_hyperband.search(X_train_scaled, y_train, epochs = 5, validation_data = (X_test_scaled, y_test))

Trial 60 Complete [00h 00m 21s]
val_accuracy: 0.7327113747596741

Best val_accuracy So Far: 0.7332944869995117
Total elapsed time: 00h 10m 25s


In [33]:
# Get best model hyperparameters
best_hyper = tuner_hyperband.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'sigmoid',
 'num_neuron': 7,
 'num_layers': 4,
 'num_layers_2': 5,
 'num_layers_3': 3,
 'num_layers_4': 5,
 'num_layers_5': 5,
 'num_layers_6': 5,
 'num_layers_7': 7,
 'tuner/epochs': 4,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 1,
 'tuner/round': 0}

In [36]:
# Evaluate best model against full test data
best_model = tuner_hyperband.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f" loss : {round(model_loss,1)*100}%, accuray : {round(model_accuracy,2)*100}%")

268/268 - 1s - loss: 0.5769 - accuracy: 0.7333 - 709ms/epoch - 3ms/step
 loss : 60.0%, accuray : 73.0%


In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
model_nn = tf.keras.models.Sequential()

# First hidden layer
model_nn.add(Dense(units = 80, activation = "sigmoid", input_dim = X_train.shape[1]))

# Second hidden layer
model_nn.add(Dense(units = 60, activation="sigmoid"))

# Third hidden layer
model_nn.add(Dense(units = 40, activation="sigmoid"))

# Fourth hidden layer
model_nn.add(Dense(units = 20, activation="sigmoid"))

# Fifth hidden layer
model_nn.add(Dense(units = 5, activation="sigmoid"))
                         
# Output layer
model_nn.add(Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
model_nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 80)                3520      
                                                                 
 dense_9 (Dense)             (None, 60)                4860      
                                                                 
 dense_10 (Dense)            (None, 40)                2440      
                                                                 
 dense_11 (Dense)            (None, 20)                820       
                                                                 
 dense_12 (Dense)            (None, 5)                 105       
                                                                 
 dense_13 (Dense)            (None, 1)                 6         
                                                                 
Total params: 11751 (45.90 KB)
Trainable params: 11751

In [25]:
# Compile 
model_nn.compile(loss = "binary_crossentropy", optimizer= "adam", metrics = ["accuracy"])

In [26]:
# Train
model_nn.fit(X_train_scaled, y_train, epochs = 10)

Epoch 1/10
804/804 [==============================] - 4s 3ms/step - loss: 0.6136 - accuracy: 0.6785
Epoch 2/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5775 - accuracy: 0.7290
Epoch 3/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5761 - accuracy: 0.7285
Epoch 4/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5752 - accuracy: 0.7294
Epoch 5/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5739 - accuracy: 0.7298
Epoch 6/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5727 - accuracy: 0.7303
Epoch 7/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5718 - accuracy: 0.7313
Epoch 8/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5704 - accuracy: 0.7301
Epoch 9/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5653 - accuracy: 0.7301
Epoch 10/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5596 - accuracy: 0.7300

In [42]:
# Callback to save the Keras model 
ModelCheckpoint("/deep-learning-challenge/checkpoint",
               save_freq='epoch'
               )

In [30]:
# Evaluate
model_loss, model_accuracy = model_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f" loss : {round(model_loss,1)*100}%, accuray : {round(model_accuracy,2)*100}%")

268/268 - 0s - loss: 0.5596 - accuracy: 0.7291 - 482ms/epoch - 2ms/step
 loss : 60.0%, accuray : 73.0%


In [ ]:
# Export our model to HDF5 file
# model_nn.save("AlphabetSoupCharity.keras")